In [ ]:
import torch
import torch.optim as optim
from get_loader import get_loader
from torchvision import transforms
from Model import EncodertoDecoder
import torch.nn as nn
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from utils import  print_examples
from Model import EncoderCNN

def train():

    transform = transforms.Compose([transforms.Resize((350,350)),
                                transforms.RandomCrop((256,256)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    images_path , caption_path = r"D:\ML\Korea\Jishu\Jishu\rsicd\images" , r"D:\ML\Korea\Jishu\Jishu\rsicd\captions.csv"
    
    BATCH_SIZE = 32
    data_loader , dataset = get_loader(images_path,caption_path ,transform,batch_size = BATCH_SIZE,num_workers=4)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_epochs = 15
    learning_rate = 3e-4
    trg_vocab_size = len(dataset.vocab)

    embedding_size = 512
    num_heads = 8
    num_decoder_layers = 4
    dropout = 0.10
    pad_idx=dataset.vocab.stoi["<PAD>"]
    save_model = True
    writer =SummaryWriter("runs/loss_plot")
    step = 0

    model = EncodertoDecoder(embeding_size=embedding_size,
                            trg_vocab_size=trg_vocab_size,
                            num_heads=num_heads,
                            num_decoder_layers=num_decoder_layers,
                            dropout=dropout, train_cnn=True).to(device)
    optimizer = optim.Adam(model.parameters(),lr = learning_rate)

    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


    for epoch in range(num_epochs):
        
        print(f"[Epoch {epoch} / {num_epochs}]")
        
        model.train()
        total_loss = 0.0
        for idx, (images, captions) in tqdm(enumerate(data_loader), total=len(data_loader), leave=False):
            images = images.to(device)
            captions = captions.to(device)
            
            output = model(images, captions[:-1])
            output = output.reshape(-1, output.shape[2])
            target = captions[1:].reshape(-1)
            
            optimizer.zero_grad()
            loss = criterion(output,target)
            lossofepoch = loss.item()
            total_loss += lossofepoch
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
            
            optimizer.step()
            writer.add_scalar("Training Loss",loss,global_step=step)
            step+=1
            
        print("Loss of the epoch is", total_loss / len(data_loader))
        torch.save(model , 'model.pth')
            
        model.eval()
        print_examples(model, device, dataset)        

In [ ]:
train()

c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
d:\ML\Korea\Jishu\Jishu\Final_Docs\Original_Image_Captioning_Model\Model.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/

[Epoch 0 / 15]
Example 1 CORRECT:  <SOS> a small area of lake is located near an overpass . <EOS> 




c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Example 1 OUTPUT: catene catene catene catene catene catene where catene where catene catene catene catene catene catene where catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene catene where catene catene catene catene catene catene catene catene catene catene
Example 1 CORRECT:  <SOS> green vegetation is planted on the hill . <EOS> 


Example 1 OUTPUT: u u u u u u u u u wetland u wetland u wetland u wetland u wetland u wetland u wetland u u wetland u wetland u wetland u wetland u wetland u wetland u wetland u u u u u wetland u u u u wetland u wetland


Loss of the epoch is 2.160471480310616
Example 1 CORRECT:  <SOS> a baseball field with one basketball field and two tennis courts next to it is beside some buildings . <EOS> 


Example 1 OUTPUT: many green trees and some buildings are in a medium residential area . <EOS>
Example 1 CORRECT:  <SOS> three white storage tanks are near a piece of green meadows . <EOS> 


Example 1 OUTPUT: many green trees and some buildings are in a medium residential area . <EOS>
[Epoch 1 / 15]
Example 1 CORRECT:  <SOS> a football field and a broad road surrounded . <EOS> 


Example 1 OUTPUT: many green trees and some buildings are in a medium residential area . <EOS>
Example 1 CORRECT:  <SOS> it 's a big piece of mounds with white snow . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school with a playground and a playground . <EOS>


Loss of the epoch is 1.7853703566000496
Example 1 CORRECT:  <SOS> two white storage tanks are semi - surrounded by green plants . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>
Example 1 CORRECT:  <SOS> some buildings are near a viaduct with many green trees . <EOS> 


Example 1 OUTPUT: many green trees are around a pond . <EOS>
[Epoch 2 / 15]
Example 1 CORRECT:  <SOS> is a square , road , trees and pastures . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>
Example 1 CORRECT:  <SOS> several buildings and green trees are around a stadium . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>


Loss of the epoch is 1.6549256872123397
Example 1 CORRECT:  <SOS> a stadium with some cars parked outside is near a river . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>
Example 1 CORRECT:  <SOS> some planes are close to a large building at an airport . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>
[Epoch 3 / 15]
Example 1 CORRECT:  <SOS> what stand beside this straight road is several airplanes with red <UNK> <UNK> . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>
Example 1 CORRECT:  <SOS> there are dark green farmland with some ponds . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>


 60%|██████    | 767/1278 [2:16:11<1:04:31,  7.58s/it]

In [4]:
train()

c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\codes\anaconda3\envs\ml\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is F

[Epoch 0 / 15]
Example 1 CORRECT:  <SOS> on the beach there is a high rise resort building with a garden and swimming pools . <EOS> 


Example 1 OUTPUT: them them while shallow stipes cruciform cruciform cruciform cruciform cruciform cruciform houses houses houses houses houses houses houses houses houses houses houses houses houses houses houses houses houses houses quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular quadrangular passes shallow shallow shallow shallow shallow shallow shallow
Example 1 CORRECT:  <SOS> it 's a big piece of agricultural land . <EOS> 


Example 1 OUTPUT: them multilateral them multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilateral multilat

Loss of the epoch is 2.162415687373733
Example 1 CORRECT:  <SOS> it 's a piece of yellow mountain with some snow . <EOS> 


Example 1 OUTPUT: many green trees are in a piece of forest . <EOS>
Example 1 CORRECT:  <SOS> a playground is surrounded by some green trees and many buildings . <EOS> 


Example 1 OUTPUT: a playground is surrounded by many green trees and many buildings . <EOS>
[Epoch 1 / 15]
Example 1 CORRECT:  <SOS> there is a right bridge across the great river . <EOS> 


Example 1 OUTPUT: a playground with a football field in it is surrounded by many green trees and buildings . <EOS>
Example 1 CORRECT:  <SOS> a basketball field and a tennis court are surrounded by many green plants . <EOS> 


Example 1 OUTPUT: many green trees and a pond are in a park . <EOS>


Loss of the epoch is 1.6807251366464557
Example 1 CORRECT:  <SOS> the building is high and low , different sizes . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>
Example 1 CORRECT:  <SOS> five baseball fields are near several green trees and a road . <EOS> 


Example 1 OUTPUT: a playground is surrounded by many green trees and many buildings . <EOS>
[Epoch 2 / 15]
Example 1 CORRECT:  <SOS> many buildings in different sizes are in an industrial area . <EOS> 


Example 1 OUTPUT: many buildings are in a commercial area . <EOS>
Example 1 CORRECT:  <SOS> many ripples are in a piece of khaki desert . <EOS> 


Example 1 OUTPUT: it is a piece of irregular khaki bareland . <EOS>


Loss of the epoch is 1.5229897575572435
Example 1 CORRECT:  <SOS> a pond with a narrow circular white bank is sandwiched between a forest and a grey white <UNK> . <EOS> 


Example 1 OUTPUT: a large number of trees are planted on both sides of the river . <EOS>
Example 1 CORRECT:  <SOS> there is a large park next to the wide road . <EOS> 


Example 1 OUTPUT: many buildings are around a park with many green trees and a pond . <EOS>
[Epoch 3 / 15]
Example 1 CORRECT:  <SOS> this densely populated area is a residential area . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a dense residential area . <EOS>
Example 1 CORRECT:  <SOS> a playground with two basketball fields next to it is surround by some green trees . <EOS> 


Example 1 OUTPUT: many pieces of agricultural land are together . <EOS>


...

Loss of the epoch is 1.2662161997692871
Example 1 CORRECT:  <SOS> many large trees are planted around the baseball field . <EOS> 


Example 1 OUTPUT: a large number of trees were planted around the lake . <EOS>
Example 1 CORRECT:  <SOS> many green trees and some buildings are in a medium residential area . <EOS> 


Example 1 OUTPUT: a large number of trees were planted around the house . <EOS>
[Epoch 11 / 15]
Example 1 CORRECT:  <SOS> a bridge is on a river with several buildings on one side of it . <EOS> 


Example 1 OUTPUT: a large number of trees were planted around the factory . <EOS>
Example 1 CORRECT:  <SOS> many buildings are on two sides of a railway station . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>


Loss of the epoch is 1.2454321216995914
Example 1 CORRECT:  <SOS> many buildings and green trees are located in a dense residential area . <EOS> 


Example 1 OUTPUT: many green trees are around a circle square . <EOS>
Example 1 CORRECT:  <SOS> a playground and a building are near railways . <EOS> 


Example 1 OUTPUT: a large number of trees are planted on both sides of the road . <EOS>
[Epoch 12 / 15]
Example 1 CORRECT:  <SOS> many green trees are in two sides of a curved small river . <EOS> 


Example 1 OUTPUT: many green trees are in a forest . <EOS>
Example 1 CORRECT:  <SOS> a playground is semi - surrounded by some green trees and many buildings . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are located in a commercial area . <EOS>


Loss of the epoch is 1.2014114070386395
Example 1 CORRECT:  <SOS> two white planes parked at the airport . <EOS> 


Example 1 OUTPUT: many buildings and some green trees are in a school . <EOS>
Example 1 CORRECT:  <SOS> it 's a piece of irregular khaki mountain . <EOS> 


Example 1 OUTPUT: many green trees are around a square with green meadows . <EOS>
[Epoch 13 / 15]
Example 1 CORRECT:  <SOS> there is a church building in the middle of the circle , parking in the parking lot . <EOS> 


Example 1 OUTPUT: many green trees are around a square . <EOS>
Example 1 CORRECT:  <SOS> a playground and two basketball fields are surrounded by several buildings and two roads . <EOS> 


Example 1 OUTPUT: many green trees and some buildings are in a school . <EOS>


Loss of the epoch is 1.1855509250843095
Example 1 CORRECT:  <SOS> on one side of the viaduct there is a bare land while on the other are some trees and grassy ground . <EOS> 


Example 1 OUTPUT: many green trees are around a circle square . <EOS>
Example 1 CORRECT:  <SOS> yellow beach is near a large piece of ocean . <EOS> 


Example 1 OUTPUT: a piece of ocean is near a yellow beach . <EOS>
[Epoch 14 / 15]
Example 1 CORRECT:  <SOS> there is a huge rectangular building located in the <UNK> of white rails . <EOS> 


Example 1 OUTPUT: many buildings are located on two sides of a railway station . <EOS>
Example 1 CORRECT:  <SOS> some scarce plants are in a bare piece of land . <EOS> 


Example 1 OUTPUT: it is a piece of yellow desert . <EOS>


Loss of the epoch is 1.1531013954599885
Example 1 CORRECT:  <SOS> the green area is surrounded by houses and is sandwiched in the middle of the road . <EOS> 


Example 1 OUTPUT: many green trees and some buildings are in a resort with a pond . <EOS>
Example 1 CORRECT:  <SOS> the port is embraced with dark green water while the industrial zone is occupied . <EOS> 


Example 1 OUTPUT: many green trees are around a square of circle . <EOS>
